In [1]:
!pip install folium

In [2]:
!pip install openpyxl

In [ ]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl import Workbook
import re


filePath = r'고등교육기관 주소록(2024.4.1.).xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl')
df_from_excel.columns = df_from_excel.loc[4].tolist()
df_from_excel = df_from_excel.drop(index=list(range(0,5)))


url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   #도로명주소
road_type2 = 'PARCEL' #지번주소
address = '&address='
keys = '&key='
primary_key = 'C6FD65EB-CF0B-3082-8DD3-756D2649B1F5'

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y


try:
    wb = load_workbook(r"학교주소좌표.xlsx", data_only=True)
    sheet  = wb.active
except:
    wb = Workbook()
    sheet = wb.active

university_list = df_from_excel['학교명'].to_list()
address_list = df_from_excel['주소'].to_list()

for num,value in enumerate(address_list):
    addr = re.sub(r'\([^)]*\)', '', value)
    print(addr)
    x,y = request_geo(addr)
    sheet.append([university_list[num],addr,x,y])

wb.save(r"학교주소좌표.xlsx")

In [ ]:
import folium

map = folium.Map(location=[37,127],zoom_start=7)

marker = folium.Marker([35.182955197, 126.878670165],
                    popup='한국폴리텍V대학 광주캠퍼스', 
                    icon = folium.Icon(color='pink'))

marker.add_to(map) 

map.save(r'uni_map.html')

In [ ]:
# 자료의 모든 대학교 주소에 마커 표시하는 코드 만들기

import pandas as pd
import folium

filePath = r'학교주소좌표.xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl',header=None)

df_from_excel.columns = ['학교이름','주소','x','y']

name_list = df_from_excel['학교이름'].to_list()
addr_list = df_from_excel['주소'].to_list()
position_x_list = df_from_excel['x'].to_list()
position_y_list = df_from_excel['y'].to_list()

map = folium.Map(location=[37,127],zoom_start=7)

for i in range(len(name_list)):
    if position_x_list[i] != 0:
        marker = folium.Marker([position_y_list[i],position_x_list[i]],
                            popup=name_list[i], 
                            icon = folium.Icon(color='blue'))
        marker.add_to(map) 

map.save(r'uni_map2.html')

In [ ]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl import Workbook
import re

filePath = r'지역별 폭염일수.xlsx'

df_from_excel = pd.read_excel(filePath, engine='openpyxl')
df_from_excel.columns = ['이름', '경도', '위도', '폭염일수']
df_from_excel = df_from_excel.drop(index=0)

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   # 도로명주소
road_type2 = 'PARCEL' # 지번주소
address = '&address='
keys = '&key='
primary_key = 'C6FD65EB-CF0B-3082-8DD3-756D2649B1F5'

def request_geo(road):
    page = requests.get(url + params + road_type + address + road + keys + primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y

try:
    wb = load_workbook(r"폭염.xlsx", data_only=True)
    sheet = wb.active
except FileNotFoundError:
    wb = Workbook()
    sheet = wb.active

location_list = df_from_excel['이름'].to_list()
heat_days_list = df_from_excel['폭염일수'].to_list()

for num, location in enumerate(location_list):
    addr = re.sub(r'\([^)]*\)', '', location)
#     print(addr)
    x, y = request_geo(addr)
    sheet.append([addr, x, y, heat_days_list[num]])

wb.save(r"폭염.xlsx")

In [ ]:
import pandas as pd
import folium

filePath = r'폭염.xlsx'
df_from_excel = pd.read_excel(filePath,engine='openpyxl',header=None)

df_from_excel.columns = ['이름','경도','위도','폭염일수']

name_list = df_from_excel['이름'].to_list()
position_x_list = df_from_excel['경도'].to_list()
position_y_list = df_from_excel['위도'].to_list()

map = folium.Map(location=[37,127],zoom_start=7)

for i in range(len(name_list)):
    if position_x_list[i] != 0:
        marker = folium.Marker(
            [position_y_list[i], position_x_list[i]],
            popup=name_list[i],
            icon=folium.Icon(color='blue')
        )
        marker.add_to(map)


map.save(r'uni_map3.html')